In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests


In [16]:
cereb= pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Cerebrospinal\Cereb_all3Ready.xlsx", header=0, index_col="Gene names")
cereb.head()

,ALS1,ALS2,ALS3,ALS4,ALS5,ALS6,ALS7,ALS8,ALS9,ALS10,...,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,20.206094,21.410808,21.007578,21.264327,20.798380,20.992680,21.190622,21.320584,20.546365,21.410149,...,20.818119,20.273776,20.588362,21.060379,21.133772,20.965446,20.451709,20.966534,20.490263,20.060107
A2M,21.227334,21.249911,21.699414,21.960486,21.265000,21.720119,21.590817,21.263972,21.954615,21.692515,...,22.186419,21.778048,22.309693,21.657449,21.197974,21.796780,21.577323,21.833239,21.480636,20.696583
ACTA2,16.702237,17.192062,16.491254,17.378139,16.681020,17.123035,17.417775,17.029531,16.079918,17.211069,...,17.634727,17.748372,17.155662,16.419831,16.378891,16.457520,17.393623,17.474386,16.848049,16.757891
ACTB,17.618774,17.832814,17.176628,18.033188,17.289458,17.800963,18.185593,17.617973,17.047510,18.100027,...,18.621876,18.413215,17.941070,17.052103,16.826727,17.093865,17.975174,18.211211,17.672492,17.438928
ADAMTS1,19.458371,20.387953,19.768644,20.161563,20.244672,20.317348,18.438870,20.458225,19.760081,20.574536,...,19.945111,19.517460,18.179602,20.783510,20.189611,20.148868,20.599072,20.158527,19.890073,18.699291


In [17]:
cereb.shape

(791, 67)

In [18]:
#Columns belonging to Group 1 and 2
ALS_columns = [col for col in cereb.columns if "ALS" in col]
Control_columns = [col for col in cereb.columns if "HC" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als =cereb[cereb[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filt = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [19]:
cont_filt.shape

(791, 67)

In [20]:
cont_filt.replace(0, np.nan, inplace=True)

In [24]:
# Automatically identify group columns by their prefixes
group1= cont_filt.loc[:, cont_filt.columns.str.startswith('HC')]
group2= cont_filt.loc[:, cont_filt.columns.str.startswith('ALS')]

#stat, p_value = mannwhitneyu(group1.iloc[0], group2.iloc[0], alternative='two-sided')

stats = []
p_vals = []
for i in range(group1.shape[0]):
    stat, p_value = mannwhitneyu(group1.iloc[i], group2.iloc[i], alternative='two-sided', method ="exact")
    stats.append(stat)
    p_vals.append(p_value)
    


In [26]:
# Add the results DataFrame to the original data DataFrame
cont_filt['U-stat'] = stats
cont_filt['p_value'] = p_vals
cont_filt.head()

,ALS1,ALS2,ALS3,ALS4,ALS5,ALS6,ALS7,ALS8,ALS9,ALS10,...,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,U-stat,p_value
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,20.206094,21.410808,21.007578,21.264327,20.798380,20.992680,21.190622,21.320584,20.546365,21.410149,...,20.588362,21.060379,21.133772,20.965446,20.451709,20.966534,20.490263,20.060107,271.0,0.005840
A2M,21.227334,21.249911,21.699414,21.960486,21.265000,21.720119,21.590817,21.263972,21.954615,21.692515,...,22.309693,21.657449,21.197974,21.796780,21.577323,21.833239,21.480636,20.696583,516.0,0.536058
ACTA2,16.702237,17.192062,16.491254,17.378139,16.681020,17.123035,17.417775,17.029531,16.079918,17.211069,...,17.155662,16.419831,16.378891,16.457520,17.393623,17.474386,16.848049,16.757891,485.0,0.843825
ACTB,17.618774,17.832814,17.176628,18.033188,17.289458,17.800963,18.185593,17.617973,17.047510,18.100027,...,17.941070,17.052103,16.826727,17.093865,17.975174,18.211211,17.672492,17.438928,500.0,0.688465
ADAMTS1,19.458371,20.387953,19.768644,20.161563,20.244672,20.317348,18.438870,20.458225,19.760081,20.574536,...,18.179602,20.783510,20.189611,20.148868,20.599072,20.158527,19.890073,18.699291,470.0,1.000000


In [28]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(cont_filt['p_value'], method='fdr_bh')
cont_filt['p_adjusted_BH'] = p_adjusted
cont_filt.head()

,ALS1,ALS2,ALS3,ALS4,ALS5,ALS6,ALS7,ALS8,ALS9,ALS10,...,HC14,HC15,HC16,HC17,HC18,HC19,HC20,U-stat,p_value,p_adjusted_BH
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,20.206094,21.410808,21.007578,21.264327,20.798380,20.992680,21.190622,21.320584,20.546365,21.410149,...,21.060379,21.133772,20.965446,20.451709,20.966534,20.490263,20.060107,271.0,0.005840,NaN
A2M,21.227334,21.249911,21.699414,21.960486,21.265000,21.720119,21.590817,21.263972,21.954615,21.692515,...,21.657449,21.197974,21.796780,21.577323,21.833239,21.480636,20.696583,516.0,0.536058,NaN
ACTA2,16.702237,17.192062,16.491254,17.378139,16.681020,17.123035,17.417775,17.029531,16.079918,17.211069,...,16.419831,16.378891,16.457520,17.393623,17.474386,16.848049,16.757891,485.0,0.843825,NaN
ACTB,17.618774,17.832814,17.176628,18.033188,17.289458,17.800963,18.185593,17.617973,17.047510,18.100027,...,17.052103,16.826727,17.093865,17.975174,18.211211,17.672492,17.438928,500.0,0.688465,NaN
ADAMTS1,19.458371,20.387953,19.768644,20.161563,20.244672,20.317348,18.438870,20.458225,19.760081,20.574536,...,20.783510,20.189611,20.148868,20.599072,20.158527,19.890073,18.699291,470.0,1.000000,NaN


In [29]:
_, p_adjusted, _, _ = multipletests(cont_filt['p_value'], method='bonferroni')
cont_filt['p_adjusted_bonf'] = p_adjusted
cont_filt.head()

,ALS1,ALS2,ALS3,ALS4,ALS5,ALS6,ALS7,ALS8,ALS9,ALS10,...,HC15,HC16,HC17,HC18,HC19,HC20,U-stat,p_value,p_adjusted_BH,p_adjusted_bonf
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,20.206094,21.410808,21.007578,21.264327,20.798380,20.992680,21.190622,21.320584,20.546365,21.410149,...,21.133772,20.965446,20.451709,20.966534,20.490263,20.060107,271.0,0.005840,NaN,1.0
A2M,21.227334,21.249911,21.699414,21.960486,21.265000,21.720119,21.590817,21.263972,21.954615,21.692515,...,21.197974,21.796780,21.577323,21.833239,21.480636,20.696583,516.0,0.536058,NaN,1.0
ACTA2,16.702237,17.192062,16.491254,17.378139,16.681020,17.123035,17.417775,17.029531,16.079918,17.211069,...,16.378891,16.457520,17.393623,17.474386,16.848049,16.757891,485.0,0.843825,NaN,1.0
ACTB,17.618774,17.832814,17.176628,18.033188,17.289458,17.800963,18.185593,17.617973,17.047510,18.100027,...,16.826727,17.093865,17.975174,18.211211,17.672492,17.438928,500.0,0.688465,NaN,1.0
ADAMTS1,19.458371,20.387953,19.768644,20.161563,20.244672,20.317348,18.438870,20.458225,19.760081,20.574536,...,20.189611,20.148868,20.599072,20.158527,19.890073,18.699291,470.0,1.000000,NaN,1.0


In [30]:
#df['average'] = df.mean(numeric_only=True, axis=1)
cont_filt['Mean Control'] = cont_filt.loc[:, cont_filt.columns.str.startswith('HC')].mean(axis=1)
cont_filt['Mean ALS'] = cont_filt.loc[:, cont_filt.columns.str.startswith('ALS')].mean(axis=1)

In [31]:
#Fold Change
cont_filt["FC ALS-C"]= cont_filt["Mean ALS"]/cont_filt["Mean Control"]

In [32]:
#Log 2 FC
cont_filt["Log2FC ALS-C"]=np.log2(cont_filt["FC ALS-C"])
cont_filt.head()

,ALS1,ALS2,ALS3,ALS4,ALS5,ALS6,ALS7,ALS8,ALS9,ALS10,...,HC19,HC20,U-stat,p_value,p_adjusted_BH,p_adjusted_bonf,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,20.206094,21.410808,21.007578,21.264327,20.798380,20.992680,21.190622,21.320584,20.546365,21.410149,...,20.490263,20.060107,271.0,0.005840,NaN,1.0,20.574704,20.871819,1.014441,0.020685
A2M,21.227334,21.249911,21.699414,21.960486,21.265000,21.720119,21.590817,21.263972,21.954615,21.692515,...,21.480636,20.696583,516.0,0.536058,NaN,1.0,21.672237,21.635717,0.998315,-0.002433
ACTA2,16.702237,17.192062,16.491254,17.378139,16.681020,17.123035,17.417775,17.029531,16.079918,17.211069,...,16.848049,16.757891,485.0,0.843825,NaN,1.0,17.093301,17.072972,0.998811,-0.001717
ACTB,17.618774,17.832814,17.176628,18.033188,17.289458,17.800963,18.185593,17.617973,17.047510,18.100027,...,17.672492,17.438928,500.0,0.688465,NaN,1.0,17.842966,17.815225,0.998445,-0.002245
ADAMTS1,19.458371,20.387953,19.768644,20.161563,20.244672,20.317348,18.438870,20.458225,19.760081,20.574536,...,19.890073,18.699291,470.0,1.000000,NaN,1.0,19.908414,19.853593,0.997246,-0.003978


In [33]:
cont_filt.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Cerebrospinal\Cereb Mann-W.xlsx")

In [34]:
# Sort the DataFrame by the absolute values of the column
down_sorted = cont_filt.loc[cont_filt["Log2FC ALS-C"].abs().sort_values(ascending=False).index]



In [35]:
down_sorted.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Cerebrospinal\Cereb Mann-W filtered.xlsx")

In [37]:
# Filter rows where 'P-value' <= 0.05
filtered_down = down_sorted[down_sorted['p_value'] <= 0.05]

In [38]:
filtered_down.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Cerebrospinal\Cereb Mann-W sorted.xlsx")